In [ ]:
# Memory 5가지 방식
# Conversation Buffer Memory - 단순히 이전 대화 내용 전체를 저장하는 메모리
# 모델 자체에는 메모리 기능이 없어서 이전 대화를 잊음
# ChatGPT는 메모리 기능이 탑재되어 모두 기억함
# 단점 : 대화의 내용이 길어질수로 메모리의 용량이 커져 비효율적이게 됨
# 사용 : text completion 할때, 예측이나 텍스트를 자동완성 하고싶을때
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True) # 챗 모델을 위한 것이면 True
memory.save_context({"input":"Hi"}, {"output":"How are you?"})
memory.load_memory_variables({})

In [6]:
# ConversationBufferWindowMemory
# 대화의 특정 부분만 저장하는 메모리
# 가장 오래된 메시지는 버려짐
# 저장범위를 설정할 수 있음
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
  return_messages=True,
  k=4
)

def add_message(input, output):
  memory.save_context({"input":input},{"output":output})
  
add_message(1, 1) # -> 버려짐
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
add_message(5, 5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [10]:
# ConversationSummaryBufferMemory
# llm을 사용하는 메모리, 메모리를 사용하는데 비용이 든다는 이야기
# mesaage 그대로 저장하는 것이 아닌 conversation의 요약을 자체적으로 해주어서 저장함
# 내용일 길면 길어질수록 효율적인 메모리
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
  memory.save_context({"input":input},{"output":output})
  
def get_history():
  return memory.load_memory_variables({})

add_message("나는 한국에 사는 쳇이야.", "멋진나라에 사는구나!")
add_message("한국은 비가 많이와", "시원하겠다")

get_history()

{'history': "The human mentions they are a chat living in Korea, and the AI responds by saying it's a wonderful country to live in. The human comments on the rainy weather in Korea, and the AI agrees that it must be refreshing."}

In [11]:
# limit이 넘으면 알아서 요약을 함
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=150,
  return_messages=True
)

def add_message(input, output):
  memory.save_context({"input":input},{"output":output})
  
def get_history():
  return memory.load_memory_variables({})

add_message("나는 한국에 사는 쳇이야.", "멋진나라에 사는구나!")
add_message("한국은 비가 많이와", "시원하겠다")

get_history()

{'history': [HumanMessage(content='나는 한국에 사는 쳇이야.'),
  AIMessage(content='멋진나라에 사는구나!'),
  HumanMessage(content='한국은 비가 많이와'),
  AIMessage(content='시원하겠다')]}

In [13]:
# ConversationKGMemory
# 대화에서 entity를 뽑아내는 메모리
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm June, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is June"})

{'history': [SystemMessage(content='On June: June lives in South Korea.')]}

In [17]:
# memory를 chain에 합치는 방법
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is June")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is June
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is June
AI: Hello June! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is June
AI: Hello June! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is June.'

In [22]:
# memory의 두가지 출력 방식 - 1. message, 2. 문자열
# 
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is June")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is June

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is June
AI: Hello June! How can I assist you today?
Human: I live in Seoul

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is June
AI: Hello June! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: What is my name?

> Finished chain.


'Your name is June. How can I assist you further, June?'

In [3]:
# RunnablePassthrough
# load_memory_variables : 메모리를 로드함
# save_context : 사람과 AI 메세지인 input, output을 메모리에 저장함
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
    
invoke_chain("My name is June")
invoke_chain("I live in Seoul")
invoke_chain("What is my name?")

content='Hello June! How can I assist you today?'
content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'
content='Your name is June. How can I assist you further, June?'


### 정리

- 기록하는 일은 Memory를 사용해서 구현해야 하는 것
- langchain
  - Off-the-shelf Chain : 자동으로 LLM으로부터 응답을 가져오고 memory를 없데이트 하는 일
  - langcahin expression : 수동으로 memory를 업데이트 하는 일
- prompt에 메모리를 추가하는 3가지 방법
  - LLM Cahin
  - Chat prompt template
  - 수동 메모리 관리